In [116]:
import os

In [117]:
%pwd
os.chdir('/Users/matt/Documents/deeplearning/')
%pwd

'/Users/matt/Documents/deeplearning'

In [118]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
  root_dir: str
  tensorboard_root_log_dir: str
  checkpoint_model_filepath: str

In [119]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [120]:


class ConfigurationManager:
  def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)
    create_directories([self.config.artifacts_root])
    
  def get_prepare_callbacks_config(self)->PrepareCallbacksConfig:
    
    config=self.config.prepare_callbacks
    checkpoint_model_dir=os.path.dirname(config.checkpoint_model_filepath)
    
    create_directories([
      checkpoint_model_dir,
      config.tensorboard_root_log_dir
    ])
    
    preare_callbacks_config=PrepareCallbacksConfig(
      root_dir=config.root_dir,
      checkpoint_model_filepath=config.checkpoint_model_filepath,
      tensorboard_root_log_dir=config.tensorboard_root_log_dir
    )
    
    return preare_callbacks_config

In [121]:
import os
import zipfile
import tensorflow as tf
from pathlib import Path
import time

In [128]:
class PrepareCallbacks:
  def __init__(self,config:PrepareCallbacksConfig):
    self.config = config
    
  @property
  def _create_tb_callbacks(self):
    timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
    tb_running_log_dir =os.path.join(
      self.config.tensorboard_root_log_dir,
      f"tb_logs_at_{timestamp}"      
    )
    return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
  
  @property
  def _create_checkpoint_callbacks(self):
    checkpoint_callbacks=tf.keras.callbacks.ModelCheckpoint(
      filepath=self.config.checkpoint_model_filepath,
      save_best_only=True)
    return checkpoint_callbacks
    
  def get_tb_ckpt_callbacks(self):
    return [
      self._create_tb_callbacks, 
      self._create_checkpoint_callbacks
      ]
                                    

In [129]:
try:
  config_manager=ConfigurationManager()
  prepare_callbacks_config= config_manager.get_prepare_callbacks_config()
  prepare_callbacks=PrepareCallbacks(config=prepare_callbacks_config)
  callbacks_list=prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e: 
  raise e

[2023-10-17 00:29:51,324: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-17 00:29:51,333: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-17 00:29:51,348: INFO: common: created directory at: artifacts]
[2023-10-17 00:29:51,351: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2023-10-17 00:29:51,358: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
